In [3]:
import torch
import pandas as pd
from pathlib import Path
from torchvision import transforms
from PIL import Image
from tqdm import tqdm
import sys
from pathlib import Path

# === Добавляем путь к корню проекта, чтобы видеть models/, data/, app/ ===
PROJECT_ROOT = Path("E:/work/mvp_ultrasound")
sys.path.append(str(PROJECT_ROOT))

from models.hybrid_unet import HybridUNetClassifierClean
from models.hybrid_unet import HybridUNetClassifierClean

# === Пути ===
ROOT = Path("E:/work/mvp_ultrasound")
CSV_PATH = ROOT / "data" / "processed" / "dataset_index.csv"
MODEL_PATH = ROOT / "models" / "mvp_model.pt"

# === Загружаем данные ===
df = pd.read_csv(CSV_PATH)
df_test = df[df['split'] == 'test']

# === Модель ===
device = torch.device("cpu")
model = HybridUNetClassifierClean(num_classes=3)
state_dict = torch.load(MODEL_PATH, map_location=device)
model.load_state_dict(state_dict, strict=False)
model.eval()

# === Трансформации ===
transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
    transforms.Normalize([0.5], [0.5])
])

# === Оценка ===
correct, total = 0, 0
for _, row in tqdm(df_test.iterrows(), total=len(df_test)):
    image = Image.open(row['path']).convert("L")
    x = transform(image).unsqueeze(0)
    with torch.no_grad():
        _, logits = model(x)
        pred = torch.argmax(logits, dim=1).item()
    if row['label'] == ["benign", "malignant", "normal"][pred]:
        correct += 1
    total += 1

accuracy = correct / total
print(f"✅ Финальная точность на тесте: {accuracy:.3f} ({correct}/{total})")

100%|████████████████████████████████████████████████████████████████████████████████| 123/123 [00:58<00:00,  2.11it/s]

✅ Финальная точность на тесте: 0.407 (50/123)
